In [22]:
import os
import numpy as np
# split full data into 2 mutually exclusive subfolders
input_dir = "./../INPUT_all/data/combined_data"
output_dir_A = "./../INPUT_all/data/combined_split_A"
output_dir_B = "./../INPUT_all/data/combined_split_B"

# Read files into a numpy array
file_list = os.listdir(input_dir)[1:]
print(file_list)
# for file in file_list
for cat in range(10):
    file = file_list[cat]
    data = np.loadtxt(os.path.join(input_dir, file), dtype=str)

    split_index = len(data) // 2
    split_A = data[:split_index]
    split_B = data[split_index:]
    if len(split_A) > len(split_B):
        split_A = split_A[:len(split_B)]
    else:
        split_B = split_B[:len(split_A)]


    np.savetxt(os.path.join(output_dir_A, f"half-images-unlabeled-{cat}.dat"), split_A, fmt="%s")
    np.savetxt(os.path.join(output_dir_B, f"half-images-unlabeled-{cat}.dat"), split_B, fmt="%s")


['full-images-unlabeled-8.dat', 'full-images-unlabeled-9.dat', 'full-images-unlabeled-4.dat', 'full-images-unlabeled-5.dat', 'full-images-unlabeled-7.dat', 'full-images-unlabeled-6.dat', 'full-images-unlabeled-2.dat', 'full-images-unlabeled-3.dat', 'full-images-unlabeled-1.dat', 'full-images-unlabeled-0.dat']


In [17]:
# Split the array into two parts

split_A.shape, split_B.shape

((3157,), (3157,))

In [21]:
# os.makedirs(output_dir_A, exist_ok=True)
# os.makedirs(output_dir_B, exist_ok=True)

)

In [ ]:


# Save the splits into separate directories
os.makedirs(output_dir_A, exist_ok=True)
os.makedirs(output_dir_B, exist_ok=True)

for i, split in enumerate([split_A, split_B], start=1):
    output_dir = output_dir_A if i == 1 else output_dir_B
    for j, array in enumerate(split):
        np.savetxt(os.path.join(output_dir, f"split{j+1}.txt"), array, fmt="%s")
